### OpenAI CliffWalking-v0 (Value Iteration and Policy Iteration)

This 

#### Importing Helper Libraries 

In [11]:
# import gymnasium 
# it generates openAI environment for RL Agent
import gymnasium as gym
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

In [18]:
environment = 'CliffWalking-v0'
renderMode = 'rgb_array'
# action mapping 
actions = {0:'UP', 1:'RIGHT', 2:'DOWN', 3:'LEFT'}

##### Agent

A generalised agent class that acts as parent class to the multiple agent classes, that act and behave differently in the environment.

In [ ]:
class Agent:

    def __init__(self) -> None:
        # this initialises the environment
        self.env = gym.make(environment,
                            render_mode = renderMode)
        self.env.reset()
    

##### Confused Agent

This agent is made such that, it selects the actions from the given action space, randomly.

In [ ]:
class ConfusedAgent(Agent):
    # This agent picks up random actions 
    # Based on the action space 
    def __init__(self) -> None:
        super().__init__()
        self.iterations = 2000

    def policy(self,env):
        # select a random action from given action space
        action = env.action_space.sample()
        return action
    
    def act(self):
        
        self.env = gym.make(environment,render_mode = renderMode)
        observation, info = self.env.reset()

        for _ in range(self.iterations):
            action = self.policy(self.env)
            observation, reward, terminated, truncated,info = self.env.step(action)

            if terminated:
                observation, info  = self.env.reset()
        self.env.close()    


##### Policy Iteration

In [ ]:
class PolicyIterationAgent(Agent):
    pass

    def PolicyEvaluation():
        pass 

    def PolicyImprovement()->bool:
        pass 

    def Run(self):

        while True: 
            # run Policy Evaluation
            
            ValueConverged = False         
            while not ValueConverged:
                ValueConverged = self.PolicyEvaluation()
            
            

            stop = self.PolicyImprovement() 

##### Value Iteration